In [1]:
DICT_FILEPATH = './data/dict.opcorpora.xml'

import xml.etree.ElementTree as ET
root = ET.parse(DICT_FILEPATH).getroot()

In [103]:
CORPUS_FILEPATH = './data/annot.opcorpora.no_ambig.xml'
corpus_root = ET.parse(CORPUS_FILEPATH).getroot()

In [64]:
from collections import defaultdict

def canonicalize_tag_name(tag_name):
    if tag_name in ['ADJF', 'ADJS', 'COMP']:
        return 'A'
    elif tag_name in ['NOUN']:
        return 'S'
    elif tag_name in ['VERB', 'INFN', 'PRTF', 'PRTS', 'GRND']:
        return 'V'
    elif tag_name in ['ADVB', 'PRCL', 'INTJ']:
        return 'ADV'
    elif tag_name in ['PREP']:
        return 'PR'
    elif tag_name in ['CONJ']:
        return 'CONJ'
    return tag_name

lemmas = root.findall('lemmata/lemma')
transformed_to_main = defaultdict(set)
main_to_tag = defaultdict(lambda: "NI")
print(len(lemmas))
for lemma in lemmas:
    main_form = lemma[0].get('t')
    main_to_tag[main_form] = canonicalize_tag_name(lemma[0][0].get('v'))
    for f in lemma[1:]:
        transformed = f.get('t')
        transformed_to_main[transformed].add(main_form)
print(len(transformed_to_main))
print(len(main_to_tag))
test_word = 'стекло'
print(transformed_to_main[test_word])
for main_form in transformed_to_main[test_word]:
    print(main_to_tag[main_form])

391243
3058733
380317
{'стекло', 'стёк'}
S
V


In [119]:
tokens = corpus_root.findall('./text/paragraphs/paragraph/sentence/tokens/token')
print(len(tokens))

92983


In [98]:
import numpy as np

def get_best_main_form(main_forms):
    if len(main_forms) == 0:
        return None
    return list(main_forms)[np.random.choice(len(main_forms), 1)[0]]

def proccess_test(test_filepath, output_filepath):
    with open(test_filepath) as test_file, open(output_filepath, 'w') as output_file:
        lines_to_write = proccess_testfile(test_file)
        print(len(lines_to_write))
        output_file.write('\n'.join(lines_to_write))

def proccess_testfile(test_file):
    lines_to_write = []
    for sentence in test_file:
        words = sentence.replace(',', '').replace('.', '').replace('?', '').replace('!', '').replace('\n', '').split(' ')
        preproccessed_words = [word.lower() for word in words]
        words_lemmatization = []
        for word in preproccessed_words:
            main_forms = transformed_to_main[word]
            main_form = get_best_main_form(main_forms)
            if main_form is None:
                main_form = word
            tag = main_to_tag[main_form]
            words_lemmatization.append((main_form, tag))
        line_to_write = ' '.join([word + '{' + word_lemmatization[0] + '=' + word_lemmatization[1] + '}' for word, word_lemmatization in zip(words, words_lemmatization)])
        lines_to_write.append(line_to_write)
    return lines_to_write

In [99]:
golden_test = ['Стала стабильнее экономическая и политическая обстановка, предприятия вывели из тени зарплаты сотрудников.\n',
               'Все Гришины одноклассники уже побывали за границей, он был чуть ли не единственным, кого не вывозили никуда дальше Красной Пахры.']
correct_result = """Стала{стать=V} стабильнее{стабильный=A} экономическая{экономический=A} и{и=CONJ} политическая{политический=A} обстановка{обстановка=S} предприятия{предприятие=S} вывели{вывести=V} из{из=PR} тени{тень=S} зарплаты{зарплата=S} сотрудников{сотрудник=S}
Все{весь=NI} Гришины{гришин=A} одноклассники{одноклассник=S} уже{уже=ADV} побывали{побывать=V} за{за=PR} границей{граница=S} он{он=NI} был{быть=V} чуть{чуть=ADV} ли{ли=ADV} не{не=ADV} единственным{единственный=A} кого{кто=NI} не{не=ADV} вывозили{вывозить=V} никуда{никуда=NI} дальше{далеко=ADV} Красной{красный=A} Пахры{Пахра=S}"""
result = '\n'.join(proccess_testfile(golden_test))
print(correct_result == result)
print()
print("RESULT")
print(result)
print()
print("EXPECTED")
print(correct_result)

False

RESULT
Стала{стал=V} стабильнее{стабильнее=A} экономическая{экономический=A} и{и=S} политическая{политический=A} обстановка{обстановка=S} предприятия{предприятие=S} вывели{вывел=V} из{из=PR} тени{теню=V} зарплаты{зарплата=S} сотрудников{сотрудник=S}
Все{весь=S} Гришины{гришины=S} одноклассники{одноклассник=S} уже{уж=S} побывали{побывал=V} за{за=PR} границей{граница=S} он{он=NPRO} был{есть=V} чуть{чуть=CONJ} ли{ли=S} не{не=ADV} единственным{единственный=A} кого{кто=NPRO} не{не=ADV} вывозили{вывожу=V} никуда{никуда=ADV} дальше{дальше=A} Красной{красная=S} Пахры{пахра=S}

EXPECTED
Стала{стать=V} стабильнее{стабильный=A} экономическая{экономический=A} и{и=CONJ} политическая{политический=A} обстановка{обстановка=S} предприятия{предприятие=S} вывели{вывести=V} из{из=PR} тени{тень=S} зарплаты{зарплата=S} сотрудников{сотрудник=S}
Все{весь=NI} Гришины{гришин=A} одноклассники{одноклассник=S} уже{уже=ADV} побывали{побывать=V} за{за=PR} границей{граница=S} он{он=NI} был{быть=V} чуть{чуть=AD

In [100]:
print(transformed_to_main['стала'])
print(transformed_to_main['стабильнее'])

{'стал'}
{'стабильнее'}


In [102]:
TEST_FILEPATH = './data/test.txt'
OUTPUT_FILEPATH = './data/output.txt'

proccess_test(TEST_FILEPATH, OUTPUT_FILEPATH)

200
